# 1 Imports

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# 2 Connect to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPEN_API_KEY')

In [3]:
openai = OpenAI()

# 3 Webpage scrapping class

In [4]:
class Website:
    """
    A utility class to represent a Website that we have scrapped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url the BeutifulSoup library
        """

        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
website = Website("https://edwarddonner.com")
print(website.title)
print(website.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

# 4 Prompt Configuration

In [6]:
# Define system prompt

system_prompt = "You are an assistant that analyzes the contets of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are lookin at a website title {website.title}"
    user_prompt += """ The contents of this website is as follows; \
        please provide a short summary of this website in markdown. \
        If it includes news or annoucements, then summrarize these too.\n\n """
    user_prompt += website.text
    return user_prompt

In [8]:
# OpenAI Message API structure

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# 5 OpenAI API Call

## 5.1 Summary Edwarddonner Website

In [9]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content       

In [10]:
output = summarize("http://edwarddonner.com")

In [11]:
# A function to display this nicely in Jupyter output, using Markdown
def display_summary(text):
    display(Markdown(text))

In [12]:
display_summary(output)

# Summary of Edward Donner's Website

Edward Donner's website serves as a platform for sharing insights and resources related to AI, particularly the use of Large Language Models (LLMs). Ed describes himself as a coder and experimentalist in the AI field, as well as a co-founder and CTO of Nebula.io, a company focused on leveraging AI for talent discovery.

## Key Sections:
- **About**: Ed provides a brief introduction about himself, mentioning his interests in coding, DJing, electronic music, and engagement with the tech community through platforms like Hacker News. He highlights his experience founding and managing AI startups, including Nebula.io and untapt.
  
- **Connect Four**: This section outlines a competitive arena concept where LLMs engage in strategic interactions against each other.

## News and Announcements:
1. **January 23, 2025**: Announcement of a hands-on LLM Workshop focused on Agents.
2. **December 21, 2024**: Welcoming a new audience segment, "SuperDataScientists."
3. **November 13, 2024**: Release of resources for "Mastering AI and LLM Engineering."
4. **October 16, 2024**: Sharing resources for transitioning from a Software Engineer to an AI Data Scientist. 

For more updates or to connect with Ed, interested parties can subscribe to his newsletter.

## 5.2 Summary of CNN Website

In [13]:
display_summary(summarize("http://cnn.com"))

# Summary of CNN's Website Content

CNN is a leading news organization providing breaking news, diverse topics, and in-depth analysis from around the world. The website covers various categories including U.S. News, World News, Politics, Business, Health, Entertainment, Sports, Science, and Climate.

## Key Announcements and Updates:
- **Digital Service Resignations**: Today, staffers from the U.S. Digital Service resigned, viewed as a mass protest related to the agency's involvement with the DOGE initiative.
- **Current Events**:
  - Republicans are seeking assistance from House leadership concerning pressures over DOGE cuts.
  - Notable developments in the Ukraine-Russia conflict and an analysis of shifting alliances in Europe due to U.S. President Trump's policies.
  - A trial related to the theft of a $6 million gold toilet has commenced.
  - Recent concerning health news highlighted, including a study suggesting antidepressants may hasten dementia decline.

Overall, CNN provides a wide range of news articles, live updates, and video content, making it a vital source for real-time information and analysis on current global issues.

## 5.3 Summary of InfoMoney Website

In [14]:
display_summary(summarize("http://infomoney.com.br"))

# InfoMoney – Informações que valem dinheiro

InfoMoney é um portal focado em fornecer informações financeiras, análises de mercado e orientações sobre investimento. O site oferece uma variedade de recursos, como planilhas de gastos e dividendos, simuladores de investimento, cursos e conteúdos educacionais sobre finanças pessoais. 

## Últimas Notícias

- **Dólar e Bitcoin**: O dólar fechou em leve baixa após atingir R$ 5,80, enquanto o Bitcoin subiu para US$ 86 mil amid concerns sobre risco.
- **Desempenho de ações**: Apesar de resultados recordes, ações da Marcopolo estão caindo. O Itaú BBA identificou 7 ações com potencial positivo no Brasil.
- **Atualizações políticas e econômicas**: Foi discutido que Lula e Bolsonaro teriam um empate técnico nas eleições, e 69% da população reconheceu a alta dos preços, culpando o governo por isso.
- **Desenvolvimentos globais**: O vídeo de IA de Trump foi exibido em um órgão do governo dos EUA, e a Casa Branca definirá quais repórteres cobrirão Trump.

## Novos Recursos e Cursos

- O site oferece uma infinidade de cursos gratuitos para investimento, como sobre renda fixa e dividendos de Fundos Imobiliários. Além disso, há uma planilha gratuita para acompanhamento de dividendos, auxiliando investidores a maximizar seus lucros.

InfoMoney se posiciona como uma fonte confiável para quem deseja melhorar sua educação financeira e acompanhar as oscilações do mercado e oportunidades de investimento.